# generate static graph

In [4]:
f = open("../../src_data/ICEWS05-15/train", encoding='utf-8')
tgraph = {}
for line in f:
    e1, r, e2, time = line.strip().split("\t")
    time = time.split("-")
    y, m, d = time
    time = m + d
    time = int(time)
    if time not in tgraph:
        tgraph[time] = {}
    if r not in tgraph[time]:
        tgraph[time][r] = []
    tgraph[time][r].append([e1, e2])
f.close()

f = open("../../src_data/ICEWS05-15/valid", encoding='utf-8')
for line in f:
    e1, r, e2, time = line.strip().split("\t")
    time = time.split("-")
    y, m, d = time
    time = m + d
    time = int(time)
    if time not in tgraph:
        tgraph[time] = {}
    if r not in tgraph[time]:
        tgraph[time][r] = []
    tgraph[time][r].append([e1, e2])
f.close()

# convert time into sorted dict
sort_t = sorted(tgraph.keys())
tg = {}
for t in sort_t:
    tg[t] = tgraph[t]
tgraph = {}
n = 0
for time in tg:
    tgraph[n] = tg[time]
    n += 1

# load the static candidate len-1 rules

In [5]:
f = open("./ICEWS05-15/ruleslen1_static.txt")
rule_1 = {}
rule_1_rev = {}
for line in f:
    body, head = line.strip().split("=>")
    he1, hr, he2 = head.strip().split("\t")
    be1, br, be2 = body.strip().split("\t")
    if hr not in rule_1:
        rule_1[hr] = {}
    if br not in rule_1_rev:
        rule_1_rev[br] = {}
    if br not in rule_1[hr]:
        rule_1[hr][br] = 0
        rule_1_rev[br][hr] = 0
    if he1 == be1:
        rule_1[hr][br] = 1
        rule_1_rev[br][hr] = 1
f.close()

# learn len-1 rules in the pattern of r1(x, y, t+T) <= r2(x, y, t)

In [6]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
window = 20
for head in rule_1:
    # rule head rel in each length-1 rule
    for body in rule_1[head]:
        # each rule body rel in rule[head]
        if body not in bn:
            bn[body] = 0.
        if body not in sd:
            sd[body] = {}
        if body not in sd_list:
            sd_list[body] = {}
        sd[body][head] = 0.
        for time_cur in range(len(tgraph)-window+1):
            if body not in tgraph[time_cur]:
                continue
            sd_list[body][head] = []
            bn[body] += len(tgraph[time_cur][body])
            for ent_pair in tgraph[time_cur][body]:
                if rule_1[head][body] == 1:
                    a, b = ent_pair
                else:
                    b, a = ent_pair
                for time_next in range(time_cur+1, time_cur+window):
                    if head not in tgraph[time_next]:
                        continue
                    for ent_pair in tgraph[time_next][head]:
                        if ent_pair[0] == a and ent_pair[1] == b and ent_pair not in sd_list[body][head]:
                            sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])            
time_end=time.time()
print('running time of len-1 temporal rule:\t',time_end-time_start)

running time of len-1 temporal rule:	 148.3582146167755


# evaluate SC of len-1 rules in the pattern of r1(x, y, t+T) <= r2(x, y, t)

In [7]:
sc_rule1_p1 = {}
cnt = 0
for head in rule_1:
    if head not in sc_rule1_p1:
        sc_rule1_p1[head] = {}
    for body in rule_1[head]:
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            sc_rule1_p1[head][body] = [rule_1[head][body], sc]
            cnt += 1
    if not sc_rule1_p1[head]:
        sc_rule1_p1.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
print("each rule and its confidence:\n", sc_rule1_p1)

The total number of rules is:	42

each rule and its confidence:
 {'Receive deployment of peacekeepers': {'Provide military protection or peacekeeping': [0, 0.2681564245810056], 'Express intent to provide military protection or peacekeeping': [0, 0.47560975609756095]}, 'Use unconventional violence': {'Use conventional military force': [0, 0.15058852081058124]}, 'Engage in diplomatic cooperation': {'Engage in diplomatic cooperation': [0, 0.13347432384129632]}, 'fight with small arms and light weapons': {'Use unconventional violence': [1, 0.12771359654690873], 'Abduct, hijack, or take hostage': [1, 0.20551005212211468], 'Physically assault': [1, 0.13851351351351351]}, "Meet at a 'third' location": {"Meet at a 'third' location": [0, 0.3311416616855947]}, 'Yield': {'Give ultimatum': [0, 0.16524216524216523]}, 'Occupy territory': {'Conduct suicide, car, or other non-military bombing': [1, 0.11587982832618025], 'Mobilize or increase armed forces': [1, 0.17423133235724744]}, 'Express intent to

# learn static len-1 rules in the pattern of r1(x, y, t) <= r2(x, y, t)

In [8]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
for body in rule_1_rev:
    # rule body rel in each length-1 rule
    if body not in bn:
        bn[body] = 0.
        sd[body] = {}
        sd_list[body] = {}
    for head in rule_1_rev[body]:
        # each head body rel in rule[body]
        sd[body][head] = 0.
        for time_cur in range(len(tgraph)):
            if body not in tgraph[time_cur]:
                continue
            sd_list[body][head] = []
            bn[body] += len(tgraph[time_cur][body])
            for ent_pair in tgraph[time_cur][body]:
                if rule_1_rev[body][head] == 1:
                    a, b = ent_pair
                else:
                    b, a = ent_pair
            
                if head not in tgraph[time_cur]:
                    continue
                for ent_pair in tgraph[time_cur][head]:
                    if ent_pair[0] == a and ent_pair[1] == b and ent_pair not in sd_list[body][head]:
                        sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])            
time_end=time.time()
print('run a head rule time:\t',time_end-time_start)

run a head rule time:	 6.649908065795898


# evaluate the SC of static len-1 rules in the pattern of r1(x, y, t) <= r2(x, y, t)

In [9]:
sc_rule1_p2 = {}
cnt = 0
for head in rule_1:
    if head not in sc_rule1_p2:
        sc_rule1_p2[head] = {}
    for body in rule_1[head]:
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            sc_rule1_p2[head][body] = [rule_1[head][body], sc]
            cnt += 1
    if not sc_rule1_p2[head]:
        sc_rule1_p2.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
print("each rule and its confidence:\n", sc_rule1_p2)

The total number of rules is:	25

each rule and its confidence:
 {'Receive deployment of peacekeepers': {'Provide military protection or peacekeeping': [0, 0.4146981627296588], 'Express intent to provide military protection or peacekeeping': [0, 0.3146067415730337]}, 'Engage in diplomatic cooperation': {'Engage in diplomatic cooperation': [0, 0.16168941979522183]}, "Meet at a 'third' location": {"Meet at a 'third' location": [0, 0.8828620888632429]}, 'Express intent to settle dispute': {'Express intent to settle dispute': [0, 0.1780929866036249], 'Demand de-escalation of military engagement': [1, 0.21428571428571427]}, 'Impose embargo, boycott, or sanctions': {'Threaten with sanctions, boycott, embargo': [1, 0.21052631578947367]}, 'Consult': {'Consult': [0, 0.42799846781280276]}, 'Make a visit': {'Host a visit': [0, 0.290575279121414]}, 'Express intent to cooperate': {'Express intent to cooperate': [0, 0.1120603015075377]}, 'Sign formal agreement': {'Sign formal agreement': [0, 0.17019

# load all the static candidate len-2 rules

In [10]:
f = open("./ICEWS05-15/ruleslen2_static.txt")
rule_2 = {}
rule_2_rev = {}
for line in f:
    body, head = line.strip().split("=>")
    he1, hr, he2 = head.strip().split("\t")
    be1, br1, be2, be3, br2, be4 = body.strip().split("\t")
    br = (br1, br2)
    if hr not in rule_2:
        rule_2[hr] = {}
    if br not in rule_2_rev:
        rule_2_rev[br] = {}
    if br not in rule_2[hr]:
        rule_2[hr][br] = 0
        rule_2_rev[br][hr] = 0
    if he1 == be1 and he1 == be3:
        rule_2[hr][br] = 1    # (a,b)<=(a,b),(a,b)
        rule_2_rev[br][hr] = 1
    elif he1 == be2 and he1 == be4:
        rule_2[hr][br] = 2    # (a,b)<=(b,a),(b,a)
        rule_2_rev[br][hr] = 2
    elif he1 == be2 and he1 == be3:
        rule_2[hr][br] = 3    # (a,b)<=(b,a),(a,b)
        rule_2_rev[br][hr] = 3
    elif he1 == be1 and he1 == be4:
        rule_2[hr][br] = 4    # (a,b)<=(a,b),(b,a)
        rule_2_rev[br][hr] = 4
    elif he1 == be1 and he2 == be4:
        rule_2[hr][br] = 5    # (a,b)<=(a,e),(e,b)
        rule_2_rev[br][hr] = 5
    elif he1 == be2 and he2 == be4:
        rule_2[hr][br] = 6    # (a,b)<=(e,a),(e,b)
        rule_2_rev[br][hr] = 6
    elif he1 == be2 and he2 == be3:
        rule_2[hr][br] = 7    # (a,b)<=(e,a),(b,e)
        rule_2_rev[br][hr] = 7
    elif he1 == be4 and he2 == be1:
        rule_2[hr][br] = 8    # (a,b)<=(b,e),(e,a)
        rule_2_rev[br][hr] = 8
    elif he1 == be3 and he2 == be2:
        rule_2[hr][br] = 9    # (a,b)<=(e,b),(a,e)
        rule_2_rev[br][hr] = 9
    elif he1 == be4 and he2 == be2:
        rule_2[hr][br] = 10    # (a,b)<=(e,b),(e,a)
        rule_2_rev[br][hr] = 10
    elif he1 == be1 and he2 == be3:
        rule_2[hr][br] = 11    # (a,b)<=(a,e),(b,e)
        rule_2_rev[br][hr] = 11
    elif he1 == be3 and he2 == be1:
        rule_2[hr][br] = 12    # (a,b)<=(b,e),(a,e)
        rule_2_rev[br][hr] = 12
    else:
        print(line)
f.close()

# learn len-2 rules in the pattern of r1(h1, t1, t+T2) <= r2(h2, t2, t) ^ r3(h3, t3, t+T1), which associated with the following cases:
    1    # (a,b)<=(a,b),(a,b)
    2    # (a,b)<=(b,a),(b,a)
    3    # (a,b)<=(b,a),(a,b)
    4    # (a,b)<=(a,b),(b,a)
    5    # (a,b)<=(a,e),(e,b)
    6    # (a,b)<=(e,a),(e,b)
    7    # (a,b)<=(e,a),(b,e)
    8    # (a,b)<=(b,e),(e,a)
    9    # (a,b)<=(e,b),(a,e)
    10   # (a,b)<=(e,b),(e,a)
    11   # (a,b)<=(a,e),(b,e)
    12   # (a,b)<=(b,e),(a,e)

In [11]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
bn_list = {}    # body number of entity pair list
window = 30
for body in rule_2_rev:
    #print("rule body:\t", body)
    # rule body rel in each length-1 rule
    if body not in bn:
        bn[body] = 0.
        #bn_list[body] = []
        sd[body] = {}
        sd_list[body] = {}
    for head in rule_2_rev[body]:
        # each head body rel in rule[body]
        sd[body][head] = 0.
        #sd_list[body][head] = []
        for time_cur in range(len(tgraph)-2*window):
            if body[0] not in tgraph[time_cur]:
                continue
            bn_list[body] = []
            sd_list[body][head] = []
            for ent_pair in tgraph[time_cur][body[0]]:
                r1h, r1t = ent_pair
                for time_next1 in range(time_cur+1, time_cur+window):
                    if body[1] not in tgraph[time_next1]:
                        continue
                    #sd_list[body][head] = []
                    #if 
                    #bn[body] += len(tgraph[time_cur][body])
                    for ent_pair in tgraph[time_next1][body[1]]:
                        r2h, r2t = ent_pair
                        if rule_2_rev[body][head] == 1 or rule_2_rev[body][head] == 2:
                            if r2h != r1h or r2t != r1t:
                                continue                                
                        elif rule_2_rev[body][head] == 4 or rule_2_rev[body][head] == 3:
                            if r2h != r1t or r2t != r1h:
                                continue
                        elif rule_2_rev[body][head] == 5 or rule_2_rev[body][head] == 8:
                            if r2h != r1t:
                                continue
                        elif rule_2_rev[body][head] == 6 or rule_2_rev[body][head] == 10:
                            if r2h != r1h:
                                continue
                        elif rule_2_rev[body][head] == 7 or rule_2_rev[body][head] == 9:
                            if r2t != r1h:
                                continue
                        elif rule_2_rev[body][head] == 11 or rule_2_rev[body][head] == 12:
                            if r2t != r1t:
                                continue
                        else:
                            print("extra pattern")
                        if [r1h, r2t] not in bn_list[body]:
                            bn_list[body].append([r1h, r2t])
                        for time_next2 in range(time_next1+1, time_next1+window):
                            if head not in tgraph[time_next2]:
                                continue                            
                            for ent_pair in tgraph[time_next2][head]:
                                if ent_pair[0] == r1h and ent_pair[1] == r2t and ent_pair not in sd_list[body][head]:
                                    sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])
            bn[body] += len(bn_list[body])
time_end=time.time()
print('run a head rule time:\t',time_end-time_start)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "D:\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_13032\1786656224.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "D:\lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "D:\lib\site-packages\IPython\core\ultratb.py", line 1326, in structured_traceback
    return VerboseTB.structured_traceback(
  File "D:\lib\site-packages\IPython\core\ultratb.py", line 1173, in structured_traceback
    formatted_exception = self.format_exception_as_a_whole(etype, evalue, etb, number_of_lines_of_context

### evaluate the SC of static len-2 rules in the pattern of r1(h1, t1, t+T2) <= r2(h2, t2, t) ^ r3(h3, t3, t+T1)

In [ ]:
cnt = 0
sc_rule2_p1 = {}
for head in rule_2:
    if head not in sc_rule2_p1:
        sc_rule2_p1[head] = {}
    for body in rule_2[head]:
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            cnt += 1
            sc_rule2_p1[head][body] = [rule_2[head][body], sc]
        if sc > 1.0:
            print(sc_rule2_p1[head][body])
    if not sc_rule2_p1[head]:
        sc_rule2_p1.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
#print("each rule and its confidence:\n", sc_rule2_p1)

### learn len-2 rules in the pattern of r1(h1, t1, t+T) <= r2(h2, t2, t) ^ r3(h3, t3, t)

In [ ]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
bn_list = {}    # body number of entity pair list
window = 30
for body in rule_2_rev:
    #print("rule body:\t", body)
    # rule body rel in each length-1 rule
    if body not in bn:
        bn[body] = 0.
        #bn_list[body] = []
        sd[body] = {}
        sd_list[body] = {}
    for head in rule_2_rev[body]:
        # each head body rel in rule[body]
        sd[body][head] = 0.
        #sd_list[body][head] = []
        for time_cur in range(len(tgraph)-window):
            if body[0] not in tgraph[time_cur]:
                continue
            bn_list[body] = []
            sd_list[body][head] = []
            for ent_pair in tgraph[time_cur][body[0]]:
                r1h, r1t = ent_pair
                if body[1] not in tgraph[time_cur]:
                    continue
                for ent_pair in tgraph[time_cur][body[1]]:
                    r2h, r2t = ent_pair
                    if rule_2_rev[body][head] == 1 or rule_2_rev[body][head] == 2:
                        if r2h != r1h or r2t != r1t:
                            continue                                
                    elif rule_2_rev[body][head] == 4 or rule_2_rev[body][head] == 3:
                        if r2h != r1t or r2t != r1h:
                            continue
                    elif rule_2_rev[body][head] == 5 or rule_2_rev[body][head] == 8:
                        if r2h != r1t:
                            continue
                    elif rule_2_rev[body][head] == 6 or rule_2_rev[body][head] == 10:
                        if r2h != r1h:
                            continue
                    elif rule_2_rev[body][head] == 7 or rule_2_rev[body][head] == 9:
                        if r2t != r1h:
                            continue
                    elif rule_2_rev[body][head] == 11 or rule_2_rev[body][head] == 12:
                            if r2t != r1t:
                                continue
                    else:
                        print("extra pattern")
                    if [r1h, r2t] not in bn_list[body]:
                        bn_list[body].append([r1h, r2t])
                            
                    for time_next in range(time_cur+1, time_cur+window):
                        if head not in tgraph[time_next]:
                            continue                            
                        for ent_pair in tgraph[time_next][head]:
                            if ent_pair[0] == r1h and ent_pair[1] == r2t and ent_pair not in sd_list[body][head]:
                                sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])
            bn[body] += len(bn_list[body])
time_end=time.time()
print('run a head rule time:\t',time_end-time_start)

### evaluate the SC of static len-2 rules in the pattern of r1(h1, t1, t+T) <= r2(h2, t2, t) ^ r3(h3, t3, t)

In [ ]:
cnt = 0
sc_rule2_p2 = {}
for head in rule_2:
    if head not in sc_rule2_p2:
        sc_rule2_p2[head] = {}
    for body in rule_2[head]:
        if bn[body] == 0.0:
            continue
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            cnt += 1
            sc_rule2_p2[head][body] = [rule_2[head][body], sc]
        if sc > 1.0:
            print(sc_rule2_p2[head][body])
    if not sc_rule2_p2[head]:
        sc_rule2_p2.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
#print("each rule and its confidence:\n", sc_rule2_p2)

### learn len-2 rules in the pattern of r1(h1, t1, t+T) <= r2(h2, t2, t) ^ r3(h3, t3, t+T)

In [ ]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
bn_list = {}    # body number of entity pair list
window = 30
for body in rule_2_rev:
    #print("rule body:\t", body)
    # rule body rel in each length-1 rule
    if body not in bn:
        bn[body] = 0.
        #bn_list[body] = []
        sd[body] = {}
        sd_list[body] = {}
    for head in rule_2_rev[body]:
        # each head body rel in rule[body]
        sd[body][head] = 0.
        #sd_list[body][head] = []
        for time_cur in range(len(tgraph)-window):
            if body[0] not in tgraph[time_cur]:
                continue
            bn_list[body] = []
            sd_list[body][head] = []
            for ent_pair in tgraph[time_cur][body[0]]:
                r1h, r1t = ent_pair
                if body[1] not in tgraph[time_cur]:
                    continue
                
                for time_next in range(time_cur+1, time_cur+window):
                    if body[1] not in tgraph[time_next]:
                        continue
                    for ent_pair in tgraph[time_next][body[1]]:
                        r2h, r2t = ent_pair
                        if rule_2_rev[body][head] == 1 or rule_2_rev[body][head] == 2:
                            if r2h != r1h or r2t != r1t:
                                continue                                
                        elif rule_2_rev[body][head] == 4 or rule_2_rev[body][head] == 3:
                            if r2h != r1t or r2t != r1h:
                                continue
                        elif rule_2_rev[body][head] == 5 or rule_2_rev[body][head] == 8:
                            if r2h != r1t:
                                continue
                        elif rule_2_rev[body][head] == 6 or rule_2_rev[body][head] == 10:
                            if r2h != r1h:
                                continue
                        elif rule_2_rev[body][head] == 7 or rule_2_rev[body][head] == 9:
                            if r2t != r1h:
                                continue
                        elif rule_2_rev[body][head] == 11 or rule_2_rev[body][head] == 12:
                            if r2t != r1t:
                                continue
                        else:
                            print("extra pattern")
                        if [r1h, r2t] not in bn_list[body]:
                            bn_list[body].append([r1h, r2t])
                            
                        if head not in tgraph[time_next]:
                            continue                            
                        for ent_pair in tgraph[time_next][head]:
                            if ent_pair[0] == r1h and ent_pair[1] == r2t and ent_pair not in sd_list[body][head]:
                                sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])
            bn[body] += len(bn_list[body])
time_end=time.time()
print('run a head rule time:\t',time_end-time_start)

### Evaluate the SC of len-2 rules in the pattern of r1(h1, t1, t+T) <= r2(h2, t2, t) ^ r3(h3, t3, t+T)

In [ ]:
cnt = 0
sc_rule2_p3 = {}
for head in rule_2:
    if head not in sc_rule2_p3:
        sc_rule2_p3[head] = {}
    for body in rule_2[head]:
        if bn[body] == 0.0:
            continue
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            cnt += 1
            sc_rule2_p3[head][body] = [rule_2[head][body], sc]
        if sc > 1.0:
            print(sc_rule2_p3[head][body])
    if not sc_rule2_p3[head]:
        sc_rule2_p3.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
#print("each rule and its confidence:\n", sc_rule2_p3)

### learn len-2 rules in the pattern of r1(h1, t1, t) <= r2(h2, t2, t) ^ r3(h3, t3, t)

In [ ]:
import time
time_start=time.time()
sd = {}   # support degree
sd_list = {}    # support degree of entity pair list
bn = {}   # body degree    standard confidence = sd / bn
bn_list = {}    # body number of entity pair list
window = 30
for body in rule_2_rev:
    #print("rule body:\t", body)
    # rule body rel in each length-1 rule
    if body not in bn:
        bn[body] = 0.
        #bn_list[body] = []
        sd[body] = {}
        sd_list[body] = {}
    for head in rule_2_rev[body]:
        # each head body rel in rule[body]
        sd[body][head] = 0.
        #sd_list[body][head] = []
        for time_cur in range(len(tgraph)):
            if body[0] not in tgraph[time_cur]:
                continue
            bn_list[body] = []
            sd_list[body][head] = []
            for ent_pair in tgraph[time_cur][body[0]]:
                r1h, r1t = ent_pair
                if body[1] not in tgraph[time_cur]:
                    continue
                                
                if body[1] not in tgraph[time_cur]:
                    continue
                for ent_pair in tgraph[time_cur][body[1]]:
                    r2h, r2t = ent_pair
                    if rule_2_rev[body][head] == 1 or rule_2_rev[body][head] == 2:
                        if r2h != r1h or r2t != r1t:
                            continue                                
                    elif rule_2_rev[body][head] == 4 or rule_2_rev[body][head] == 3:
                        if r2h != r1t or r2t != r1h:
                            continue
                    elif rule_2_rev[body][head] == 5 or rule_2_rev[body][head] == 8:
                        if r2h != r1t:
                            continue
                    elif rule_2_rev[body][head] == 6 or rule_2_rev[body][head] == 10:
                        if r2h != r1h:
                            continue
                    elif rule_2_rev[body][head] == 7 or rule_2_rev[body][head] == 9:
                        if r2t != r1h:
                            continue
                    elif rule_2_rev[body][head] == 11 or rule_2_rev[body][head] == 12:
                            if r2t != r1t:
                                continue
                    else:
                        print("extra pattern")
                    if [r1h, r2t] not in bn_list[body]:
                        bn_list[body].append([r1h, r2t])

                    if head not in tgraph[time_cur]:
                        continue                            
                    for ent_pair in tgraph[time_cur][head]:
                        if ent_pair[0] == r1h and ent_pair[1] == r2t and ent_pair not in sd_list[body][head]:
                            sd_list[body][head].append(ent_pair)
            sd[body][head] += len(sd_list[body][head])
            bn[body] += len(bn_list[body])
time_end=time.time()
print('run a head rule time:\t',time_end-time_start)

### Evaluate the SC of len-2 rules in the pattern of r1(h1, t1, t) <= r2(h2, t2, t) ^ r3(h3, t3, t)

In [ ]:
cnt = 0
sc_rule2_p4 = {}
for head in rule_2:
    if head not in sc_rule2_p4:
        sc_rule2_p4[head] = {}
    for body in rule_2[head]:
        if bn[body] == 0.0:
            continue
        sc = sd[body][head] / bn[body]
        if sc > 0.1:
            cnt += 1
            sc_rule2_p4[head][body] = [rule_2[head][body], sc]
        if sc > 1.0:
            print(sc_rule2_p4[head][body])
    if not sc_rule2_p4[head]:
        sc_rule2_p4.pop(head)
print("The total number of rules is:\t%d\n" % cnt)
#print("each rule and its confidence:\n", sc_rule2_p4)

### rule reform via rel2id

In [ ]:
from pathlib import Path
import pkg_resources
import pickle
from collections import defaultdict
from typing import Dict, Tuple, List
import os

DATA_PATH = pkg_resources.resource_filename('tkbc', 'data/')
ff = open(os.path.join(DATA_PATH, 'ICEWS05-15', 'rel_id'), 'r', encoding='utf-8')
rel_dict = {}
for line in ff:
    rel, id = line.strip().split("\t")
    rel_dict[rel] = id
ff.close()

In [ ]:
import json
rule1_p1 = {}
rule1_p1_t = {}
len_r = len(rel_dict)
for head in sc_rule1_p1:
    for body in sc_rule1_p1[head]:
        if head == body:
            break
        if int(rel_dict[head]) not in rule1_p1:
            rule1_p1[int(rel_dict[head])] = {}
            rule1_p1_t[int(rel_dict[head])] = {}
            rule1_p1[int(rel_dict[head])+len_r] = {}
            rule1_p1_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule1_p1[head][body]
        if patt_confi[0] == 1:
            rule1_p1[int(rel_dict[head])][int(rel_dict[body])] = sc_rule1_p1[head][body][1]
            rule1_p1_t[int(rel_dict[head])][int(rel_dict[body])] = sc_rule1_p1[head][body][1]
            rule1_p1[int(rel_dict[head])+len_r][int(rel_dict[body])+len_r] = sc_rule1_p1[head][body][1]
            rule1_p1_t[int(rel_dict[head])+len_r][int(rel_dict[body])+len_r] = sc_rule1_p1[head][body][1]
        else:
            rule1_p1[int(rel_dict[head])][int(rel_dict[body])+len_r] = sc_rule1_p1[head][body][1]
            rule1_p1_t[int(rel_dict[head])][int(rel_dict[body])+len_r] = sc_rule1_p1[head][body][1]
            rule1_p1[int(rel_dict[head])+len_r][int(rel_dict[body])] = sc_rule1_p1[head][body][1]
            rule1_p1_t[int(rel_dict[head])+len_r][int(rel_dict[body])] = sc_rule1_p1[head][body][1]

In [ ]:
rule1_p2 = {}
rule1_p2_t = {}
for head in sc_rule1_p2:
    for body in sc_rule1_p2[head]:
        if head == body:
            break
        if int(rel_dict[head]) not in rule1_p2:
            rule1_p2[int(rel_dict[head])] = {}
            rule1_p2_t[int(rel_dict[head])] = {}
            rule1_p2[int(rel_dict[head])+len_r] = {}
            rule1_p2_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule1_p2[head][body]
        if patt_confi[0] == 1:
            rule1_p2[int(rel_dict[head])][int(rel_dict[body])] = sc_rule1_p2[head][body][1]
            rule1_p2_t[int(rel_dict[head])][int(rel_dict[body])] = sc_rule1_p2[head][body][1]
            rule1_p2[int(rel_dict[head])+len_r][int(rel_dict[body])+len_r] = sc_rule1_p2[head][body][1]
            rule1_p2_t[int(rel_dict[head])+len_r][int(rel_dict[body])+len_r] = sc_rule1_p2[head][body][1]
        else:
            rule1_p2[int(rel_dict[head])][int(rel_dict[body])+len_r] = sc_rule1_p2[head][body][1]
            rule1_p2_t[int(rel_dict[head])][int(rel_dict[body])+len_r] = sc_rule1_p2[head][body][1]
            rule1_p2[int(rel_dict[head])+len_r][int(rel_dict[body])] = sc_rule1_p2[head][body][1]
            rule1_p2_t[int(rel_dict[head])+len_r][int(rel_dict[body])] = sc_rule1_p2[head][body][1]

In [ ]:
for head in rule1_p1:
    for body in rule1_p1[head]:
        if head in rule1_p2:
            if body in rule1_p2[head]:
                if rule1_p1[head][body] > rule1_p2[head][body]:
                    rule1_p2_t[head].pop(body)
                else:
                    rule1_p1_t[head].pop(body)

In [ ]:
for head in list(rule1_p1_t.keys()):
    if not rule1_p1_t[head]:
        del rule1_p1_t[head]
for head in list(rule1_p2_t.keys()):
    if not rule1_p2_t[head]:
        rule1_p2_t.pop(head)

In [ ]:
with open('ICEWS05-15/rule1_p1.json', 'w') as f:
    json.dump(rule1_p1_t, f)
with open('ICEWS05-15/rule1_p2.json', 'w') as f:
    json.dump(rule1_p2_t, f)

### The patterns of len-2 rules:
    1    # (a,b)<=(a,b),(a,b)
    2    # (a,b)<=(b,a),(b,a)
    3    # (a,b)<=(b,a),(a,b)
    4    # (a,b)<=(a,b),(b,a)
    5    # (a,b)<=(a,e),(e,b)
    6    # (a,b)<=(e,a),(e,b)
    7    # (a,b)<=(e,a),(b,e)
    8    # (a,b)<=(b,e),(e,a)
    9    # (a,b)<=(e,b),(a,e)
    10   # (a,b)<=(e,b),(e,a)
    11   # (a,b)<=(a,e),(b,e)
    12   # (a,b)<=(b,e),(a,e)

In [ ]:
import json
rule2_p1 = {}
rule2_p1_t = {}
for head in sc_rule2_p1:
    for body in sc_rule2_p1[head]:
        if int(rel_dict[head]) not in rule2_p1:
            rule2_p1[int(rel_dict[head])] = {}
            rule2_p1_t[int(rel_dict[head])] = {}
            rule2_p1[int(rel_dict[head])+len_r] = {}
            rule2_p1_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule2_p1[head][body]
        if patt_confi[0] == 1 or patt_confi[0] == 5 or patt_confi[0] == 9:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 2 or patt_confi[0] == 7 or patt_confi[0] == 8:
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 3 or patt_confi[0] == 6 or patt_confi[0] == 12:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 4 or patt_confi[0] == 10 or patt_confi[0] == 11:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]

In [ ]:
rule2_p2 = {}
rule2_p2_t = {}
for head in sc_rule2_p2:
    for body in sc_rule2_p2[head]:
        if int(rel_dict[head]) not in rule2_p2:
            rule2_p2[int(rel_dict[head])] = {}
            rule2_p2_t[int(rel_dict[head])] = {}
            rule2_p2[int(rel_dict[head])+len_r] = {}
            rule2_p2_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule2_p2[head][body]
        if patt_confi[0] == 1 or patt_confi[0] == 5 or patt_confi[0] == 9:
            rule2_p2[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p2_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p2[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p2_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 2 or patt_confi[0] == 7 or patt_confi[0] == 8:
            rule2_p2[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p2_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p2[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p2_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 3 or patt_confi[0] == 6 or patt_confi[0] == 12:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 4 or patt_confi[0] == 10 or patt_confi[0] == 11:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]

In [ ]:
rule2_p3 = {}
rule2_p3_t = {}
for head in sc_rule2_p3:
    for body in sc_rule2_p3[head]:
        if int(rel_dict[head]) not in rule2_p3:
            rule2_p3[int(rel_dict[head])] = {}
            rule2_p3_t[int(rel_dict[head])] = {}
            rule2_p3[int(rel_dict[head])+len_r] = {}
            rule2_p3_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule2_p3[head][body]
        if patt_confi[0] == 1 or patt_confi[0] == 5 or patt_confi[0] == 9:
            rule2_p3[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p3_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p3[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p3_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 2 or patt_confi[0] == 7 or patt_confi[0] == 8:
            rule2_p3[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p3_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p3[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p3_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 3 or patt_confi[0] == 6 or patt_confi[0] == 12:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 4 or patt_confi[0] == 10 or patt_confi[0] == 11:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]

In [ ]:
rule2_p4 = {}
rule2_p4_t = {}
for head in sc_rule2_p4:
    for body in sc_rule2_p4[head]:
        if int(rel_dict[head]) not in rule2_p4:
            rule2_p4[int(rel_dict[head])] = {}
            rule2_p4_t[int(rel_dict[head])] = {}
            rule2_p4[int(rel_dict[head])+len_r] = {}
            rule2_p4_t[int(rel_dict[head])+len_r] = {}
        patt_confi = sc_rule2_p4[head][body]
        if patt_confi[0] == 1 or patt_confi[0] == 5 or patt_confi[0] == 9:
            rule2_p4[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p4_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p4[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p4_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 2 or patt_confi[0] == 7 or patt_confi[0] == 8:
            rule2_p4[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p4_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p4[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p4_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 3 or patt_confi[0] == 6 or patt_confi[0] == 12:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
        if patt_confi[0] == 4 or patt_confi[0] == 10 or patt_confi[0] == 11:
            rule2_p1[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])][(int(rel_dict[body[0]]), int(rel_dict[body[1]])+len_r)] = patt_confi[1]
            rule2_p1[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]
            rule2_p1_t[int(rel_dict[head])+len_r][(int(rel_dict[body[0]])+len_r, int(rel_dict[body[1]]))] = patt_confi[1]

In [ ]:
for head in list(rule2_p1.keys()):
    for body in list(rule2_p1[head].keys()):
        if head in rule2_p2:
            if body in rule2_p2[head]:
                if rule2_p1[head][body] > rule2_p2[head][body]:
                    rule2_p2[head].pop(body)
                else:
                    rule2_p1[head].pop(body)

for head in list(rule2_p1.keys()):
    for body in list(rule2_p1[head].keys()):
        if head in rule2_p3:
            if body in rule2_p3[head]:
                if rule2_p1[head][body] > rule2_p3[head][body]:
                    rule2_p3[head].pop(body)
                else:
                    rule2_p1[head].pop(body)

for head in list(rule2_p1.keys()):
    for body in list(rule2_p1[head].keys()):
        if head in rule2_p4:
            if body in rule2_p4[head]:
                if rule2_p1[head][body] > rule2_p4[head][body]:
                    rule2_p4[head].pop(body)
                else:
                    rule2_p1[head].pop(body)

for head in list(rule2_p2.keys()):
    for body in list(rule2_p2[head].keys()):
        if head in rule2_p3:
            if body in rule2_p3[head]:
                if rule2_p2[head][body] > rule2_p3[head][body]:
                    rule2_p3[head].pop(body)
                else:
                    rule2_p2[head].pop(body)                

for head in list(rule2_p2.keys()):
    for body in list(rule2_p2[head].keys()):
        if head in rule2_p4:
            if body in rule2_p4[head]:
                if rule2_p2[head][body] > rule2_p4[head][body]:
                    rule2_p4[head].pop(body)
                else:
                    rule2_p2[head].pop(body)

for head in list(rule2_p3.keys()):
    for body in list(rule2_p3[head].keys()):
        if head in rule2_p4:
            if body in rule2_p4[head]:
                if rule2_p3[head][body] > rule2_p4[head][body]:
                    rule2_p4[head].pop(body)
                else:
                    rule2_p3[head].pop(body)

In [ ]:
for head in list(rule2_p1.keys()):
    if not rule2_p1[head]:
        del rule2_p1[head]
for head in list(rule2_p2.keys()):
    if not rule2_p2[head]:
        rule2_p2.pop(head)
for head in list(rule2_p3.keys()):
    if not rule2_p3[head]:
        rule2_p3.pop(head)
for head in list(rule2_p4.keys()):
    if not rule2_p4[head]:
        rule2_p4.pop(head)

In [ ]:
f = open('ICEWS05-15/rule2_p1.txt', 'w', encoding='utf-8')
for head in rule2_p1:
    for body in rule2_p1[head]:
        f.write(str(head) + "\t" + str(body[0]) + "\t" + str(body[1]) + "\t" + str(rule2_p1[head][body]) + "\n")
f.close()
f = open('ICEWS05-15/rule2_p2.txt', 'w', encoding='utf-8')
for head in rule2_p2:
    for body in rule2_p2[head]:
        f.write(str(head) + "\t" + str(body[0]) + "\t" + str(body[1]) + "\t" + str(rule2_p2[head][body]) + "\n")
f.close()
f = open('ICEWS05-15/rule2_p3.txt', 'w', encoding='utf-8')
for head in rule2_p3:
    for body in rule2_p3[head]:
        f.write(str(head) + "\t" + str(body[0]) + "\t" + str(body[1]) + "\t" + str(rule2_p3[head][body]) + "\n")
f.close()
f = open('ICEWS05-15/rule2_p4.txt', 'w', encoding='utf-8')
for head in rule2_p4:
    for body in rule2_p4[head]:
        f.write(str(head) + "\t" + str(body[0]) + "\t" + str(body[1]) + "\t" + str(rule2_p4[head][body]) + "\n")
f.close()